In [3]:
import pandas as pd
import numpy as nu
import urllib, json
import time 
import sys

def temporary_hold(duration):
    print ("wait...")
    time.sleep(timer)
    print ("go..")

def financial_variables(stock):
    #function to query Yahoo YQL and return the financial data for a stock based on stock symbol. 
    #the variables are selected according to Altman Z-Score
    total_assets=""
    total_liability=""
    current_assets=""
    current_liability=""
    retained_earnings=""
    ebitda=""
    sales=""
    stockprice=""
    try:
        baseurl = "https://query.yahooapis.com/v1/public/yql?"
        #extract data from balance sheet 
        yql_bs_query = "select * from yahoo.finance.balancesheet where symbol in ('"+stock+"')"
        yql_bs_url = baseurl + urllib.parse.urlencode({'q':yql_bs_query}) + "&format=json&diagnostics=true&env=store%3A%2F%2Fdatatables.org%2Falltableswithkeys&callback="
        bs_json = pd.io.json.read_json(yql_bs_url)
        if bs_json["query"]["results"]["balancesheet"]["statement"] is not None:
            try:
                total_assets=bs_json["query"]["results"]["balancesheet"]["statement"][0]['TotalAssets']['content']
            except:
                total_assets=""
            try:
                total_liability=bs_json["query"]["results"]["balancesheet"]["statement"][0]['TotalLiabilities']['content']
            except:
                total_liability=""
            try:
                current_assets=bs_json["query"]["results"]["balancesheet"]["statement"][0]['TotalCurrentAssets']['content']
            except:
                current_assets=""
            try:
                current_liability=bs_json["query"]["results"]["balancesheet"]["statement"][0]['TotalCurrentLiabilities']['content']
            except:
                current_liability = ""
            try:
                retained_earnings=bs_json["query"]["results"]["balancesheet"]["statement"][0]['RetainedEarnings']['content']
            except: 
                retained_earnings=""
            
            #extract data from income statement
            yql_is_query = "select * from yahoo.finance.incomestatement where symbol in ('"+stock+"')"
            yql_is_url = baseurl + urllib.parse.urlencode({'q':yql_is_query}) + "&format=json&diagnostics=true&env=store%3A%2F%2Fdatatables.org%2Falltableswithkeys&callback="
            is_json = pd.io.json.read_json(yql_is_url)

            try:
                ebitda=is_json["query"]["results"]["incomestatement"]["statement"][0]['EarningsBeforeInterestAndTaxes']['content']
            except:
                ebitda=""
            try:
                sales=is_json["query"]["results"]["incomestatement"]["statement"][0]['GrossProfit']['content']
            except:
                sales=""
            
            #extract data from finance quotes 
            yql_qt_query = "select * from yahoo.finance.quotes where symbol in ('"+stock+"')"
            yql_qt_url = baseurl + urllib.parse.urlencode({'q':yql_qt_query}) + "&format=json&diagnostics=true&env=store%3A%2F%2Fdatatables.org%2Falltableswithkeys&callback="
            qt_json = pd.io.json.read_json(yql_qt_url)
            try:
                stockprice = qt_json["query"]["results"]["quote"]["LastTradePriceOnly"]
            except:
                stockprice = ""
    except:
        print ("error on ", stock, ". Error message",sys.exc_info())
    print ("\n",stock, "|", total_assets, "|", total_liability, "|", current_assets,"|",current_liability,"|",retained_earnings,"|",ebitda,"|",sales,"|", stockprice)
    return (stock, total_assets, total_liability, current_assets,current_liability,retained_earnings,ebitda, sales, stockprice)


#dfwithnames.to_csv('data/altman_results.csv')

In [63]:
firms_fin_location = "../data/allpubliccomp.csv"
    firms = pd.read_csv(firms_fin_location)
    print ("\nstock | total_assets | total_liability | current_assets | current_liability | retained_earnings | ebitda | sales | stockprice")
    altman_table = [financial_variables(stock.lstrip('$')) for stock in firms['Symbol'][22:24]]
    df['names']= str(firms['Name'][1:2])
    df = pd.DataFrame(data=altman_table, index=firms['Symbol'][1:3], columns=['stock', 'total_assets', 'total_liability', 'current_assets','current_liability','retained_earnings','ebitda', 'sales', 'stockprice'])
    dfwithnames.to_csv('../data/altman_results.csv')


stock | total_assets | total_liability | current_assets | current_liability | retained_earnings | ebitda | sales | stockprice



 ACTG | 548920000 | 47382000 | 248351000 | 38341000 | -111851000 | -12690000 | 37192000 | 19.12



 ACHC | 2140468000 | 1253892000 | 229840000 | 142540000 | 79313000 | 47224000 | 280417000 | 64.06
(2, 9)


In [62]:
print (df.shape)
df['names']= str(firms['Name'][1:2])

print (df.columns)

(2, 10)
Index(['stock', 'total_assets', 'total_liability', 'current_assets', 'current_liability', 'retained_earnings', 'ebitda', 'sales', 'stockprice', 'names'], dtype='object')
